In [1]:
INPUT_LOCATION = 'day4.txt'
SEPARATOR = '\n'

In [2]:
# fetch data 
raw_data = open(INPUT_LOCATION, 'r').readlines()
clean_data = list(map(lambda x: x.rstrip() if len(x) > 1 else x,raw_data))


In [3]:
import re
from functools import reduce

# parse data array into an iterator which returns a 
# dict for every accepted document separated by
# SEPARATOR:

class Parser:

    SEPARATOR = '\n'
    pattern = re.compile(r'(\w{3}):(\S*)')
    
    def validate(self,data):
        return data != SEPARATOR

    def __init__(self, p):
        self.p = p
        self.index = 0
        self.current = []
        
    def __iter__(self):
        return self
        
    def __next__(self):
        
        self.current = []
        
        if (not self.validate(self.p[self.index])):
            self.index += 1
            if self.index == len(self.p):
               raise StopIteration 
        
        while True:
            if (not self.validate(self.p[self.index])):
                break
            self.current.append(self.p[self.index])
            self.index += 1
            if self.index == len(self.p):
                raise StopIteration 
        
        list_tuples = self.pattern.findall(" ".join(self.current))
        list_dict = list( map(lambda x: {x[0]: x[1]}, list_tuples))
        res_dict = reduce(lambda x, y: {**x, **y}, list_dict)
        return  res_dict


In [4]:
 # # check data and iterator
 # passports = iter(Parser(clean_data))
 # for pas in enumerate(passports):
 #    print(pas)

In [5]:
# valid passports  (Question 1)
def valid_fields_passports(data):
    valid_pas = list()
    passports = iter(Parser(data))
    keys = [
        'byr', 
        'iyr', 
        'eyr', 
        'hgt', 
        'hcl', 
        'ecl', 
        'pid', 
        # 'cid'
        ]

   

    for index, pas in enumerate(passports):
        # print('[*KO ]::', count, index)
        if len(pas) > 6 : 
            if all (k in pas.keys() for k in keys):
                valid_pas.append(pas) 
                # print('[ OK ]::', count,index )
           
                # print('[*KO ]::', count, index)

    return valid_pas

In [44]:
# valid passports  (Question 2)
def filter_rules(passport):
    
    # height rule too long for 1 line
    hgt = False 
    height = re.search(r'([0-9]+)([a-z]{2})',passport['hgt'])
    if(height != None):
       um = height.group(2)
       height = int(height.group(1))
       if um == 'cm': hgt =  height < 193 and height > 150
       elif um == 'in': hgt =  height < 76 and height > 59 
    
      
    rules = {
        'byr': int(passport['byr']) > 1920 and int(passport['byr']) < 2008,
        'iyr': int(passport['iyr']) > 2010 and int(passport['iyr']) < 2020,
        'eyr': int(passport['eyr']) > 2020 and int(passport['eyr']) < 2030,
        'hgt': hgt,   
        'hcl': re.search(r'#[0-9a-f]{6}',passport['hcl']) != None,
        'ecl': passport['ecl'] in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'],
        'pid': re.search(r'[0-9]{9}',passport['pid']) != None,
        }
         

    if all(value == True for value in rules.values()):
        return True  
    
    return False
    

def valid_passports(data):
    return list(filter(lambda x: filter_rules(x),  valid_fields_passports(data)))

In [45]:
# count data
print(len(valid_fields_passports(clean_data)), 'passports have required fields.')
print(len(valid_passports(clean_data)), 'are valid passports.')

213 passports have required fields.
89 are valid passports.
